# Analysing the COVID-19 pandemic in Bosnia and Herzegovina

The analysie will be preformed on a dataset gathered from the <a href="https://www.who.int/">WHO</a> website. The first part of this analysis will be data cleaning, wich is the most important part of data analysis. You know how they say it if the data is not clean we get garbage in and garbage out.

The next part will contain visualizations to get a more understanding picture of the situation so we can preform some statistical methods later. 

After we finished the data cleaning and visualization process will continue on data modeling so we can make predictions in the later part when we will actualy use our data to make predictions on how the situation will improve or not in the future.

When all of this is set and done we will make the conclusion and suggest how things can be done in the future to improve the situation.

## Table of Contetn's

* [Importing the nececery Libraries](#importing-the-nececery-libraries)
* [Data import and exploration](#data-import-and-exploration)

## Importing the nececery Libraries

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

## Data import and exploration

In [2]:
rawData = pd.read_excel(os.path.join("../dataSet/rawData/", "mbih.xlsx"), engine='openpyxl')

In [3]:
rawData.head()

,Unnamed: 0,date,total_cases,new_cases,population,population_density,median_age,aged_65_older
0,6393,2020-03-05,2,2,3280815,68.496,42.5,16.569
1,6394,2020-03-06,2,0,3280815,68.496,42.5,16.569
2,6395,2020-03-07,3,1,3280815,68.496,42.5,16.569
3,6396,2020-03-08,3,0,3280815,68.496,42.5,16.569
4,6397,2020-03-09,3,0,3280815,68.496,42.5,16.569


In [4]:
rawData.info() # checking the datatype of each column, the null valuse

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          301 non-null    int64  
 1   date                301 non-null    object 
 2   total_cases         301 non-null    int64  
 3   new_cases           301 non-null    int64  
 4   population          301 non-null    int64  
 5   population_density  301 non-null    float64
 6   median_age          301 non-null    float64
 7   aged_65_older       301 non-null    float64
dtypes: float64(3), int64(4), object(1)
memory usage: 18.9+ KB


In [5]:
rawData.describe() # fast overview of statistical methods for each column

,Unnamed: 0,total_cases,new_cases,population,population_density,median_age,aged_65_older
count,301.000000,301.000000,301.000000,301.0,301.000,301.0,301.000
mean,6543.000000,26374.262458,368.720930,3280815.0,68.496,42.5,16.569
std,87.035433,33015.025192,467.075392,0.0,0.000,0.0,0.000
min,6393.000000,2.000000,0.000000,3280815.0,68.496,42.5,16.569
25%,6468.000000,2321.000000,27.000000,3280815.0,68.496,42.5,16.569
50%,6543.000000,12296.000000,220.000000,3280815.0,68.496,42.5,16.569
75%,6618.000000,32845.000000,453.000000,3280815.0,68.496,42.5,16.569
max,6693.000000,110985.000000,1953.000000,3280815.0,68.496,42.5,16.569


## Data preprocessing